In [12]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
%autoreload

from allennlp.commands.elmo import ElmoEmbedder
import os
from sys import path
path.append('../..')
from relation_extraction.data import utils
import h5py
import numpy as np
dataset = 'ddi' #TODO: update this
preprocessing_type = 'entity_blinding' #TODO: update this; mostly we are only going to be using original
cuda_device=2 #cuda device for the elmo embeddings generator
data_path = '/crimea/geeticka/data/relation_extraction/' + dataset + '/'
def res(path): return os.path.join(data_path, path)
out_path = 'pre-processed/'+ preprocessing_type +'/elmo/'

In [14]:
if not os.path.exists(res(out_path)):
    os.makedirs(res(out_path))
if not os.path.exists(res(out_path + 'input-sentences')):
    os.makedirs(res(out_path + 'input-sentences'))

Weights and options for Semeval

In [15]:
# OPTIONS_FILE = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway_5.5B/elmo_2x4096_512_2048cnn_2xhighway_5.5B_options.json"
# WEIGHT_FILE = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway_5.5B/elmo_2x4096_512_2048cnn_2xhighway_5.5B_weights.hdf5"

Weights and options for the PubMed

In [16]:
OPTIONS_FILE = 'https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/contributed/pubmed/elmo_2x4096_512_2048cnn_2xhighway_options.json'
WEIGHT_FILE = 'https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/contributed/pubmed/elmo_2x4096_512_2048cnn_2xhighway_weights_PubMed_only.hdf5'

In [17]:
elmo = ElmoEmbedder(weight_file=WEIGHT_FILE, options_file=OPTIONS_FILE, cuda_device=cuda_device)

## Getting all of the ELMo layers

In [18]:
# border_sizes = [-1, 50, 1, 20]
border_sizes = [-1] # -1 for both i2b2 and ddi

In [19]:
def get_sentences(filename, border_size):
    data = open(res('pre-processed/'+ preprocessing_type+'/' + filename))
    data = utils.split_data_cut_sentence(data, border_size)
    sentences = data[0]
    return sentences

In [20]:
def write_sentences_to_txt(sentences, filename):
    with open(res(out_path + 'input-sentences/'+filename + '.txt'), 'w') as f:
        for sentence in sentences:
            sentence_string = " ".join(sentence)
            f.write(sentence_string + "\n")

In [21]:
def write_hdf5(filename):
    with open(res(out_path + 'input-sentences/' + filename + '.txt')) as file:
        elmo.embed_file(input_file=file, 
                        output_file_path = res(out_path + filename + '.hdf5'))

In [22]:
for border_size in border_sizes:
    train_sentences = get_sentences('train_'+ preprocessing_type +'.txt', border_size)
    test_sentences = get_sentences('test_'+ preprocessing_type +'.txt', border_size)
    write_sentences_to_txt(train_sentences, 'train_'+ preprocessing_type +'_border_' + str(border_size))
    write_sentences_to_txt(test_sentences, 'test_'+ preprocessing_type +'_border_' + str(border_size))
    write_hdf5('train_'+ preprocessing_type +'_border_' + str(border_size))
    write_hdf5('test_'+ preprocessing_type +'_border_' + str(border_size))

21233it [02:43, 130.15it/s]
4696it [00:35, 131.59it/s]
